In [1]:
import imaginet.task as task
task = reload(task)
import funktional.layer as layer
from funktional.layer import params
import funktional.util as util
from funktional.util import autoassign
import theano.tensor as T
import random
import numpy

Couldn't import dot_parser, loading of dot files will not be possible.


Using gpu device 0: Tesla K20m


In [5]:
from imaginet.simple_data import *
import imaginet.data_provider as dp

In [6]:
seed = 123
if seed is not None:
    random.seed(seed)
    numpy.random.seed(seed)
prov = dp.getDataProvider('flickr8k', root='/home/gchrupala/repos/reimaginet')
data_c = SimpleData(prov, tokenize=characters, min_df=1, scale=True, batch_size=64, shuffle=True, limit=None)
data_w = SimpleData(prov, tokenize=words, min_df=10, scale=True, batch_size=64, shuffle=True, limit=None)

Could not read file /home/gchrupala/repos/reimaginet/data/flickr8k/dataset.ipa.jsonl.gz: IPA transcription not available


In [ ]:
import imaginet.corep
reload(imaginet.corep)

In [ ]:
corep = imaginet.corep.Corep(data_c, data_w, 128, 512, 512, 3, 1, 4096)

In [ ]:
from collections import Counter
import sys
def valid_loss(task, data):
        result = []
        for item in data.iter_valid_batches():
            inp, target_v, _, _ = item
            result.append(task.loss_test(inp, target_v))
        return result

things = [("word", (corep.data_w, corep.Task_w, Counter())),
          ("char", (corep.data_c, corep.Task_c, Counter())) ]
for epoch in range(1,3):
    for name, thing in things:
        data, task, costs = thing
        for _j, item in enumerate(data_c.iter_train_batches()):
                j = _j + 1
                inp, target_v, _, _ = item
                cost = task.train(inp, target_v)
                costs.update(Counter({'cost':cost, 'N':1}))
                if j % 10 == 0:
                    print epoch, name, j, j*data.batch_size, "train", "".join([str(costs['cost']/costs['N'])])
                    sys.stdout.flush()

In [ ]:
things[0][1][2]

In [ ]:
things[1][1][2]

In [ ]:
corep.save("/tmp/corep.zip")

In [17]:
import imaginet.defn.visual
reload(imaginet.defn.visual)

<module 'imaginet.defn.visual' from '/exp2/gchrupala/repos/reimaginet/imaginet/defn/visual.py'>

In [18]:
model = imaginet.defn.visual.VisualModel(data_c, 128, 256)

In [19]:
model

<imaginet.defn.visual.VisualModel instance at 0x7fb208be4ef0>

In [20]:
dir(model)

['Visual',
 '__doc__',
 '__init__',
 '__module__',
 'config',
 'data',
 'dataset',
 'depth',
 'lr',
 'max_norm',
 'params',
 'save',
 'size',
 'size_embed',
 'size_target',
 'weights']

In [ ]:
from collections import Counter
import sys
for epoch in range(1,5):
    costs = Counter()
    name = "phoneme"
    data = model.dataset
    task = model.Visual
    for _j, item in enumerate(data_c.iter_train_batches()):
                j = _j + 1
                inp, target_v, _, _ = item
                cost = task.train(inp, target_v)
                costs.update(Counter({'cost':cost, 'N':1}))
                if j % 10 == 0:
                    print epoch, name, j, j*data.batch_size, "train", "".join([str(costs['cost']/costs['N'])])
                    sys.stdout.flush()

1 phoneme 10 640 train 0.748108720779
1 phoneme 20 1280 train 0.750677633286
1 phoneme 30 1920 train 0.752432314555
1 phoneme 40 2560 train 0.752964878082
1 phoneme 50 3200 train 0.752103118896
1 phoneme 60 3840 train 0.753414471944
1 phoneme 70 4480 train 0.754182924543
1 phoneme 80 5120 train 0.754847860336
1 phoneme 90 5760 train 0.755607520209
1 phoneme 100 6400 train 0.756101379395
